In [6]:
!pip install boto3 pandas scikit-learn flask influxdb-client

In [7]:
import boto3
import pandas as pd
from io import StringIO

bucket_name = 'siotweather'
file_key = 'weather_csv.csv'

s3 = boto3.client('s3')

try:
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    data = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(data))
    print(df.head())
except Exception as e:
    print(e)

   blue  clouds device  green  humidity  luminosity  pressure  rainfall  red  \
0    15       0  ESP32     22     35.08           3   1016.71       0.0   15   
1    16       0  ESP32     23     39.12           4   1016.82       0.0   16   
2    17       0  ESP32     25     42.12           4   1016.87       0.0   17   
3    18       0  ESP32     27     41.30           4   1016.92       0.0   18   
4    20       0  ESP32     30     41.33           5   1016.96       0.0   20   

   rssi  temperature                            time  visibility  weather  
0   -42        18.40  2024-12-06T08:19:48.848604049Z       10000        0  
1   -42        16.44  2024-12-06T08:22:25.515115144Z       10000        0  
2   -42        15.36  2024-12-06T08:25:02.165617919Z       10000        0  
3   -41        15.26  2024-12-06T08:27:38.825231342Z       10000        0  
4   -42        15.07  2024-12-06T08:30:15.519030642Z       10000        0  


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Updated features list (excluding 'rssi' and other irrelevant columns)
features = ['blue', 'clouds', 'green', 'humidity', 'luminosity', 
            'pressure', 'rainfall', 'red', 'temperature', 'visibility']
target = 'weather'

# Prepare the features and target variable
X = df[features]
y = df[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Decision Tree model
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.95

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       161
           1       0.94      0.92      0.93        89

    accuracy                           0.95       250
   macro avg       0.95      0.95      0.95       250
weighted avg       0.95      0.95      0.95       250



In [9]:
import boto3
from botocore.exceptions import NoCredentialsError
import os

# Define variables
bucket_name = 'siotweather'
s3_key = 'decision_tree_model.ipynb'  # The path in the bucket
notebook_path = '/home/ubuntu/weather.ipynb'        # Local notebook path

# Create an S3 client (with IAM role permissions)
s3 = boto3.client('s3')

try:
    # Upload the notebook to S3
    s3.upload_file(notebook_path, bucket_name, s3_key)
    print(f"Successfully uploaded {notebook_path} to s3://{bucket_name}/{s3_key}")
except FileNotFoundError:
    print("The file was not found. Please check the path.")
except NoCredentialsError:
    print("Credentials not available. Check your IAM role.")

Successfully uploaded /home/ubuntu/weather.ipynb to s3://siotweather/decision_tree_model.ipynb


In [10]:
import joblib

# Save the trained Decision Tree model to a file
joblib.dump(clf, 'decision_tree_model.pkl')
print("Model saved as decision_tree_model.pkl")

Model saved as decision_tree_model.pkl


In [12]:
import boto3

# Define S3 bucket and key
bucket_name = 'siotweather'
s3_key = 'decision_tree_model.pkl'  # Path in the S3 bucket
local_model_path = 'decision_tree_model.pkl'

# Upload the model to S3
s3 = boto3.client('s3')
s3.upload_file(local_model_path, bucket_name, s3_key)

print(f"Model uploaded to s3://{bucket_name}/{s3_key}")

Model uploaded to s3://siotweather/decision_tree_model.pkl
